# MNIST Diffusion Model

In [1]:
# Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

import pickle
from tqdm import tqdm

In [2]:
# Importing data
dl_path = './MNIST'

# Run first time to download dataset
#mnist = fetch_openml('mnist_784', version=1, data_home=dl_path)

# After fetching and processing for the first time
#with open('./MNIST/mnist_processed.pkl', 'wb') as f:
#    pickle.dump(mnist, f)

# Pickled data for quick retrieval
with open('./MNIST/mnist_processed.pkl', 'rb') as f:
    mnist = pickle.load(f)

In [3]:
# Splitting into train/test data
X, y = mnist["data"], mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=10000, random_state=42, stratify=y)

X_train, X_test, y_train, y_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)

# Undersampling training set to remove sampling bias
unique, counts = np.unique(y_train, return_counts=True)
min_samples = min(counts)

sample_ids = []
for u in unique :
    class_samples = y_train[y_train == u].index.to_numpy()

    random_indices = np.random.choice(len(class_samples), min_samples, replace=False)
    sample_ids.extend(class_samples[random_indices])

X_train, y_train = X_train.iloc[sample_ids], y_train.iloc[sample_ids]

# Convert data into PyTorch tensors
X_train_tensor = torch.tensor(X_train.values/255.0, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.astype(np.int64))
X_test_tensor = torch.tensor(X_test.values/255.0, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.astype(np.int64))

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class Diffusor(nn.module) :
    def __init__(self) :